In [ ]:
import torch
import torch.nn as nn
import numpy as np

# --- 1. Thiết lập Tham số và Đầu vào ---

# Input: x = 5.1
input_x = 5.1

# Hidden Layer: 1 lớp ẩn chứa 2 nơ-ron
w_hidden = torch.tensor(
    [[0.5], [-0.5]], dtype=torch.float32
)  # (2, 1) -> W[0] là 0.5, W[1] là -0.5
b_hidden = torch.tensor([1.0, -1.0], dtype=torch.float32)  # Bias cho H1 và H2

# Output Layer: 1 nơ-ron
w_output = torch.tensor([0.3, -0.2], dtype=torch.float32)  # Trọng số nhận từ H1 và H2
b_output = torch.tensor(0.5, dtype=torch.float32)  # Bias cho O

# Chuyển input x thành tensor (1, 1)
x = torch.tensor([input_x], dtype=torch.float32).unsqueeze(0)  # Shape: (1, 1)

# --- 2. Định nghĩa Mô hình MLP (Dùng hàm tính toán thủ công) ---


def forward_pass(x):
    # --- Lớp Ẩn (Hidden Layer) ---

    # 1. Tính tổng có trọng số (z = x*W + b).
    # Do input x là scalar, ta thực hiện tính toán từng nơ-ron:

    # Z_H1 = x * w_hidden[0] + b_hidden[0]
    z_h1 = x * w_hidden[0] + b_hidden[0]  # (5.1 * 0.5) + 1.0 = 2.55 + 1.0 = 3.55

    # Z_H2 = x * w_hidden[1] + b_hidden[1]
    z_h2 = x * w_hidden[1] + b_hidden[1]  # (5.1 * -0.5) + (-1.0) = -2.55 - 1.0 = -3.55

    # Gom lại thành tensor (1, 2)
    z_hidden = torch.cat((z_h1, z_h2), dim=1)

    # 2. Áp dụng Hàm kích hoạt ReLU
    a_hidden = nn.functional.relu(z_hidden)  # ReLU(3.55) = 3.55; ReLU(-3.55) = 0.0

    # a_hidden = [3.55, 0.0]

    # --- Lớp Đầu ra (Output Layer) ---

    # 1. Tính tổng có trọng số (z = A_hidden * W_output + b_output)
    # W_output phải được reshape để nhân ma trận: (2,) -> (2, 1)

    # Z_O = (A_hidden[0] * w_output[0]) + (A_hidden[1] * w_output[1]) + b_output
    z_output = torch.dot(a_hidden.squeeze(0), w_output) + b_output

    # Z_O = (3.55 * 0.3) + (0.0 * -0.2) + 0.5
    # Z_O = 1.065 + 0.0 + 0.5 = 1.565

    # 2. Áp dụng Hàm kích hoạt Sigmoid
    output = torch.sigmoid(z_output)

    return output


# --- 3. Thực hiện Dự đoán và Kết luận ---

prediction_tensor = forward_pass(x)
prediction_value = prediction_tensor.item()

print(f"1. Đầu ra sau lớp Ẩn (ReLU): [3.55, 0.0]")
print(f"2. Đầu vào cho Sigmoid (Z_output): 1.565")
print(f"3. Đầu ra cuối cùng (Sigmoid): {prediction_value:.4f}")
print("---")

if prediction_value > 0.5:
    result = "Loại 1"
else:
    result = "Loại 0"

print(f"Vì giá trị {prediction_value:.4f} > 0.5, mô hình phân loại thuộc: {result}")

1. Đầu ra sau lớp Ẩn (ReLU): [3.55, 0.0]
2. Đầu vào cho Sigmoid (Z_output): 1.565
3. Đầu ra cuối cùng (Sigmoid): 0.8271
---
Vì giá trị 0.8271 > 0.5, mô hình phân loại thuộc: Loại 1


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 1. Định nghĩa Cấu trúc Mô hình MLP ---


class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        # 1. Định nghĩa Hidden Layer: Input 1, Output 2
        self.linear_hidden = nn.Linear(1, 2)
        # 2. Định nghĩa Output Layer: Input 2, Output 1
        self.linear_output = nn.Linear(2, 1)

    def forward(self, x):
        # Hidden Layer: Linear -> ReLU
        x = self.linear_hidden(x)
        x = F.relu(x)

        # Output Layer: Linear -> Sigmoid
        x = self.linear_output(x)
        x = torch.sigmoid(x)

        # Squeeze để loại bỏ chiều 1 (ví dụ: [1, 1] -> [1])
        return x.squeeze(1)


# --- 2. Khởi tạo và Đặt Trọng số ---

model = SimpleMLP()

# Trọng số và Bias từ đề bài:
# Hidden: w=[0.5, -0.5], b=[1, -1]
# Output: w=[0.3, -0.2], b=[0.5]

# Thiết lập trọng số cho Hidden Layer
with torch.no_grad():
    # .weight phải là (out_features, in_features) -> (2, 1)
    model.linear_hidden.weight = nn.Parameter(
        torch.tensor([[0.5], [-0.5]], dtype=torch.float32)
    )
    model.linear_hidden.bias = nn.Parameter(
        torch.tensor([1.0, -1.0], dtype=torch.float32)
    )

    # Thiết lập trọng số cho Output Layer
    # .weight phải là (out_features, in_features) -> (1, 2)
    model.linear_output.weight = nn.Parameter(
        torch.tensor([[0.3, -0.2]], dtype=torch.float32)
    )
    model.linear_output.bias = nn.Parameter(torch.tensor([0.5], dtype=torch.float32))

# --- 3. Dự đoán và Kết luận ---

# Input x = 5.1 (cần có shape [Batch_size, Input_features] -> [1, 1])
input_x = torch.tensor([[5.1]], dtype=torch.float32)

# Chuyển mô hình sang chế độ đánh giá
model.eval()

with torch.no_grad():
    prediction_tensor = model(input_x)

prediction_value = prediction_tensor.item()

print(f"Đầu ra cuối cùng (Sigmoid): {prediction_value:.4f}")
print("---")

if prediction_value > 0.5:
    result = "Loại 1"
else:
    result = "Loại 0"

print(f"Kết quả phân loại (> 0.5 là Loại 1): {result}")

Đầu ra cuối cùng (Sigmoid): 0.8271
---
Kết quả phân loại (> 0.5 là Loại 1): Loại 1


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 1. Định nghĩa Cấu trúc Mô hình bằng nn.Sequential() ---

model = nn.Sequential(
    # Lớp Ẩn: Linear (1 input -> 2 output)
    nn.Linear(1, 2),
    # Hàm kích hoạt ReLU
    nn.ReLU(),
    # Lớp Đầu ra: Linear (2 input -> 1 output)
    nn.Linear(2, 1),
)

# --- 2. Khởi tạo và Đặt Trọng số ---

# Trọng số và Bias từ đề bài:
w_hidden = torch.tensor([[0.5], [-0.5]], dtype=torch.float32)  # (2, 1)
b_hidden = torch.tensor([1.0, -1.0], dtype=torch.float32)
w_output = torch.tensor([[1.5, 2]], dtype=torch.float32)  # (1, 2)
b_output = torch.tensor([0.0], dtype=torch.float32)  # (1,)

with torch.no_grad():
    # Lớp Ẩn là phần tử thứ 0 của Sequential
    model[0].weight = nn.Parameter(w_hidden)
    model[0].bias = nn.Parameter(b_hidden)

    # Lớp Đầu ra là phần tử thứ 2 của Sequential
    model[2].weight = nn.Parameter(w_output)
    model[2].bias = nn.Parameter(b_output)

# --- 3. Dự đoán và Kết luận ---

input_x = torch.tensor([[50.0]], dtype=torch.float32)
model.eval()

with torch.no_grad():
    prediction_tensor = model(input_x)

prediction_value = prediction_tensor.item()

print(f"Đầu ra cuối cùng (Sigmoid): {prediction_value:.4f}")
print("---")


Đầu ra cuối cùng (Sigmoid): 39.0000
---


In [ ]:
model = nn.Sequential(
    nn.Linear(784, 128), nn.ReLU(), nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 10)
)

W3 = list(model.parameters())[4]
print(W3.shape)

torch.Size([10, 64])


In [8]:
y_pred = torch.tensor([[2.0, 1.0, 0.1]])
y_true = torch.tensor([0])

criterion = nn.CrossEntropyLoss()
print(criterion(y_pred, y_true))

tensor(0.4170)
